# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

In [2]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = '../data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

5 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



In [4]:
# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
train_num = train_Y.shape[0]
df.head()

,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500


# 作業1
* 試著在補空值區塊, 替換並執行兩種以上填補的缺值, 看看何者比較好?

In [7]:
# 空值補 -1, 做羅吉斯迴歸
df_m1 = df.fillna(-1)
train_X = df_m1[:train_num]
estimator = LogisticRegression()
result1 = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
print(f'The score when we fill -1 to NAN : {result1}')

df_m2 = df.fillna(0)
train_X2 = df_m2[:train_num]
result2 = cross_val_score(estimator, train_X2, train_Y, cv=5).mean()
print(f'The score when we fill 0 to NAN : {result2}')

df_m3 = df.fillna(df.median())
train_X3 = df_m3[:train_num]
result3 = cross_val_score(estimator, train_X3, train_Y, cv=5).mean()
print(f'The score when we fill Median to NAN : {result3}')

df_m4 = df.fillna(df.mean())
train_X4 = df_m4[:train_num]
result4 = cross_val_score(estimator, train_X4, train_Y, cv=5).mean()
print(f'The score when we fill Mean to NAN : {result4}')

The score when we fill -1 to NAN : 0.6960299128976762
The score when we fill 0 to NAN : 0.6971535084032942
The score when we fill Median to NAN : 0.6992934218081011
The score when we fill Mean to NAN : 0.6981761033723469


由上可知由中位數替代缺失值得表現最好

# 作業2
* 使用不同的標準化方式 ( 原值 / 最小最大化 / 標準化 )，搭配羅吉斯迴歸模型，何者效果最好?

In [10]:
df = df.fillna(df.mean())
train_X1 = df[:train_num]
result5 = cross_val_score(estimator, train_X1, train_Y, cv=5).mean()
print(f'The score of original data : {result5}')
      
df_mm = MinMaxScaler().fit_transform(df)
train_X2 = df_mm[:train_num]
result6 = cross_val_score(estimator, train_X2, train_Y, cv=5).mean()
print(f'The score of applying MinMaxScaler : {result6}')

df_ss = StandardScaler().fit_transform(df)
train_X3 = df_ss[:train_num]
result7 = cross_val_score(estimator, train_X3, train_Y, cv=5).mean()
print(f'The score of applying StandardScaler : {result7}')

The score of original data : 0.6981761033723469
The score of applying MinMaxScaler : 0.6993501991462476
The score of applying StandardScaler : 0.6959413955734954


原值搭配Logistic Regression 表現最好